In [9]:
import sys
sys.path.append('models')
sys.path.append('instances_&_dict')

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import EncoderEEG
import warnings
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [3]:
torch.cuda.is_available()
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU Name: NVIDIA GeForce MX450


In [4]:
dataset=torch.load(r"C:\Users\hp\Desktop\M.Tech\ML\ML_Project\Project Dataset\eeg_signals_raw_with_mean_std.pth")
labels=dataset['labels']
print(labels)

['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']


In [5]:
data=dataset['dataset']
len(data)

11965

In [6]:
def Data(data,subject_no=0):
    if subject_no!=0:
        x=[data[i]['eeg'][:,20:460] for i in range(len(data)) if data[i]['subject']==subject_no]
        
        y=[data[i]['label'] for i in range(len(data)) if data[i]['subject']==subject_no]
    
    else:
        x=[data[i]['eeg'][:,20:460] for i in range(len(data))]

        
        y=[data[i]['label'] for i in range(len(data))]

        
    return x,y

In [7]:
x,y=Data(data,subject_no=0)
# Ensure that x and y are lists of tensors
x = [torch.tensor(item, dtype=torch.float32) if not isinstance(item, torch.Tensor) else item for item in x]
y = [torch.tensor(item, dtype=torch.long) if not isinstance(item, torch.Tensor) else item for item in y]

# Stack the tensors
x_tensor = torch.stack(x)
y_tensor = torch.stack(y)

print(f"x_tensor shape: {x_tensor.shape}")
print(f"y_tensor shape: {y_tensor.shape}")

x_tensor shape: torch.Size([11965, 128, 440])
y_tensor shape: torch.Size([11965])


In [12]:
model=torch.load(r'C:\Users\hp\Desktop\M.Tech\ML\ML_Project\ML_Project-main\instances_&_dict\EEGEncoder.pth')
model=model.to(device)

In [13]:
subject_data = {}

# Iterate through each subject (1 to 6)
for subject_no in range(1, 7):
    x, y = Data(data, subject_no=subject_no)  # Get EEG data and labels for the subject
    # Debugging step: Check the type and shape of x and y
    print(f"Subject {subject_no} - Type of x: {type(x)}, Shape of x: {len(x)}, Type of y: {type(y)}, Shape of y: {len(y)}")
    
    # If x is a list of tensors (e.g., list of EEG data arrays), stack them
    x = [torch.tensor(item, dtype=torch.float32) if not isinstance(item, torch.Tensor) else item for item in x]
    y = [torch.tensor(item, dtype=torch.long) if not isinstance(item, torch.Tensor) else item for item in y]

    # Stack the tensors
    x_tensor = torch.stack(x)
    y_tensor = torch.stack(y) # Assuming classification task, use long for labels

    x_tensor = x_tensor.to(device)  # Move input tensor to the same device (GPU)
    y_tensor = y_tensor.to(device)  
    # Check if x_tensor is of the correct shape
    print(f"Subject {subject_no} - x_tensor shape: {x_tensor.shape}")
    print(f"Subject {subject_no} - y_tensor shape: {y_tensor.shape}")

    # Create a DataLoader for batching (if needed)
    dataset = TensorDataset(x_tensor, y_tensor)
    loader = DataLoader(dataset, batch_size=16, shuffle=False)
    
    all_embeddings = []
    
    # Disable gradients since we are not training the model
    with torch.no_grad():
        for batch_x, batch_y in loader:
            # Forward pass through the model
            output, embeddings = model(batch_x)  # This gives the model's output (e.g., logits, embeddings)
            all_embeddings.append(embeddings)  # Save the output for each batch
    
    # Concatenate all the outputs into a single tensor (for the entire subject)
    subject_output = torch.cat(all_embeddings, dim=0)  # Concatenate along batch dimension
    
    # Save EEG data, labels, and model output (embeddings) for the subject
    subject_data[subject_no] = {
        'EEG': x_tensor,
        'Labels': y_tensor,
        'z': subject_output  # Store the output (e.g., embeddings, logits, etc.)
    }

Subject 1 - Type of x: <class 'list'>, Shape of x: 1985, Type of y: <class 'list'>, Shape of y: 1985
Subject 1 - x_tensor shape: torch.Size([1985, 128, 440])
Subject 1 - y_tensor shape: torch.Size([1985])
Subject 2 - Type of x: <class 'list'>, Shape of x: 1996, Type of y: <class 'list'>, Shape of y: 1996
Subject 2 - x_tensor shape: torch.Size([1996, 128, 440])
Subject 2 - y_tensor shape: torch.Size([1996])
Subject 3 - Type of x: <class 'list'>, Shape of x: 1996, Type of y: <class 'list'>, Shape of y: 1996
Subject 3 - x_tensor shape: torch.Size([1996, 128, 440])
Subject 3 - y_tensor shape: torch.Size([1996])
Subject 4 - Type of x: <class 'list'>, Shape of x: 1996, Type of y: <class 'list'>, Shape of y: 1996
Subject 4 - x_tensor shape: torch.Size([1996, 128, 440])
Subject 4 - y_tensor shape: torch.Size([1996])
Subject 5 - Type of x: <class 'list'>, Shape of x: 1996, Type of y: <class 'list'>, Shape of y: 1996
Subject 5 - x_tensor shape: torch.Size([1996, 128, 440])
Subject 5 - y_tensor s

In [17]:
torch.save(subject_data,'data_with_embeddings.pth')

In [18]:
data_for_GAN = torch.load('data_with_embeddings.pth')

In [19]:
data_for_GAN[1]['z']

tensor([[-0.5167, -0.3976, -0.0349,  ..., -0.8051,  0.5117, -0.7190],
        [ 1.3497, -0.1515, -0.0863,  ..., -0.9664, -1.3735,  0.4914],
        [ 0.4486,  0.1777, -0.7065,  ...,  0.1288, -0.2138,  0.5208],
        ...,
        [-0.2847,  0.0429,  0.2183,  ...,  0.6406,  0.1834, -0.1103],
        [ 1.1469, -0.3220,  0.2021,  ..., -2.3834, -1.2500,  0.1670],
        [-0.2633, -0.1413, -0.9956,  ...,  0.6392,  0.5316,  0.0058]],
       device='cuda:0')